# TD Ameritrade – Python API
The TD Ameritrade API allows users to easily execute trades and pull both fundamental and technical data on all public equity and debt securities.  It also offers data on publicly traded derivatives such as stock options, index & commodity futures, and interest rate swaps.

### Set Up
1.	Go to https://developer.tdameritrade.com/
2.	Click “Register”
3.	Fill in requested fields
4.	Once Registered, click on “My Apps”
5.	Select “Add a new App”
6.	Fill out the fields as follows 
7.	Once the app is approved, click on the app name and navigate to the “KEYS” tab: 
8.	Now that your app is registered copy the “Key” from the above section, and head over to the included Jupyter notebook to test out some of the functionality of the API.

### Python Set up for the API
Now that your app is registered, you will be able to pull a wealth of info from the API.
#### Needed Packages
1.	requests
2.	pandas
3.	time
4.  matplotlib
5.  numpy
6.  seaborn
7.  statsmodels.api
8.  math
9. statistics

#### Needed Configuration Inputs
1.	Ticker file path.  This is a CSV file that you will create that has the set of tickers you wish to analyze.  Place this file in your download's folder.
2.	API Key: pulled from the developer page
3.	Define these required configurations in the Jupyter notebook:
  

### Data & Analysis

1. Downloading fundamental data
2. Downloading pricing data
3. Calculating Annualized Returns
4. Running a Regression Model 


In [ ]:
######################
###Libraries & Paths##
######################

import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import math
import statistics

## Snagging Downloaded Files

The "get_download_path" function allows python to find your downloads folder's filepath and use it pull open your ticker list csv file.  The output of the below cell should be a list of the tickers you inputted in the ticker list csv file.


In [ ]:
def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')

ticker_path = get_download_path()+'/Ticker_List.csv'
key = ''  ##PUT YOUR KEY HERE

tickers_file = pd.read_csv(ticker_path, header=None)
tickers = []
for ticker in tickers_file.values:
    tickers.append(ticker[0])
print(tickers)

## Manual Edits
Uncomment out the below cell and edit the list's content to replace the tickers you wish to evaluate.

In [ ]:
#tickers
tickers = ['AAPL']


## Defined Functions
While this code block does not run anything on its own, it holds functions that will be used later in the program.  The TD Ameritrade API contains functions that don't always output data in the most clean (useable) format, so we have developed some functions that "scrub" the output so that it returns in a better format.  Each function will be examined more thouroughly as we go through the lecture, but they will allow us to analyze a public stock from both a quantitative and qualitative perspective.

In [ ]:
def get_price_hist(ticker,period,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'peirod':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['ticker'] = ticker
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        return hist_data
    except:
        df = pd.DataFrame()
        return df
    

def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df

def scrub_fundamental_data(tickers,key):
    master = pd.DataFrame()
    count = 1
    for ticker in tickers:
        temp = get_fundamental_from_td(ticker,key)
        temp = pd.DataFrame(temp).T
        hist_data = get_annual_returns([ticker],key)
        try:
            temp.columns = ['beta', 'bookValuePerShare', 'currentRatio', 'divGrowthRate3Year',
           'dividendAmount', 'dividendDate', 'dividendPayAmount',
           'dividendPayDate', 'dividendYield', 'epsChange', 'epsChangePercentTTM',
           'epsChangeYear', 'epsTTM', 'grossMarginMRQ', 'grossMarginTTM', 'high52',
           'interestCoverage', 'low52', 'ltDebtToEquity', 'marketCap',
           'marketCapFloat', 'netProfitMarginMRQ', 'netProfitMarginTTM',
           'operatingMarginMRQ', 'operatingMarginTTM', 'pbRatio', 'pcfRatio',
           'peRatio', 'pegRatio', 'prRatio', 'quickRatio', 'returnOnAssets',
           'returnOnEquity', 'returnOnInvestment', 'revChangeIn', 'revChangeTTM',
           'revChangeYear', 'sharesOutstanding', 'shortIntDayToCover',
           'shortIntToFloat', 'ticker', 'totalDebtToCapital', 'totalDebtToEquity',
           'vol10DayAvg', 'vol1DayAvg', 'vol3MonthAvg']
            temp = pd.merge(temp, hist_data, on='ticker')
        except:
            continue
        
        master = master.append(temp).reset_index(drop=True)
        count+= 1
    return master

def scrub_price_hist(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        price_data = get_price_hist(ticker,1,key)
        master = master.append(price_data, ignore_index=True)
    return master

def get_annual_returns(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        try:
            price_data = get_price_hist(ticker,1,key).iloc[::-1]
            return_percent = price_data['close'].iloc[0] / price_data['close'].iloc[-1] -1
            df = pd.DataFrame({"ticker":ticker,  
                               "Annual_Return_Percent":return_percent},
                               index = [0])
            master = master.append(df, ignore_index=True)
        except:

            print('error with', ticker)
    return master

## Preliminary Stock analysis
This section will show you how to calculate a stocks beta relative to a market proxy using 1 year daily data.  When
picking securities for yours or another's portolio, it is first helpful to know the risk tolerance of the investmentor
relative to the risk of investing in the broad market.  Beta will allow us to estimate this for any number of stocks, then
make relative conclusions. 


In [ ]:
# This cell will get the price history for our short list.
aapl_data = scrub_price_hist(tickers,key)
aapl_data.head()

In [ ]:
#To make analysis easier let's only look at the close and date and sort the date in descending order
aapl_data = aapl_data[['close','datetime','ticker']]
aapl_data = aapl_data.sort_values(by = 'datetime', ascending = False)
aapl_data.head()

In [ ]:
#Add a new column to show daily returns 
aapl_data['AAPL_Return_%'] = aapl_data['close'] / aapl_data['close'].shift(-1) - 1
aapl_data.rename(columns={'close': 'AAPL_Close'}, inplace=True)
aapl_data.drop(columns = ['ticker'], inplace = True)
aapl_data.head()

In [ ]:
# When starting an analysis of a stock, it is helpful to understand how the security performs relative to the market.
# To accomplish this, we will use the S&P 500 as a proxy for the market, and calculate the beta of the stock relative 
# to the proxy.  This cell replicates the above for ticker SPY which is one of the most liquid ETFs the tracks the S&P 500
spy_data = scrub_price_hist(['SPY'],key)
spy_data = spy_data[['close','datetime','ticker']]
spy_data = spy_data.sort_values(by = 'datetime', ascending = False)
spy_data['SPY_Return_%'] = spy_data['close'] / spy_data['close'].shift(-1) - 1
spy_data.rename(columns={'close': 'SPY_Close'}, inplace=True)
spy_data.drop(columns = ['ticker'], inplace = True)
spy_data.head()

In [ ]:
#Now we'll merge the two sets
consol_data = pd.merge(aapl_data,spy_data,on=['datetime'], how = 'left').dropna() 
#note that you need to dropna because of how we calc'd returns, the last date in the set will not have a return value 

consol_data.head()


In [ ]:
##Now calc beta using statsmodels
x = consol_data[['SPY_Return_%']]
y = consol_data[['AAPL_Return_%']]

model = sm.OLS(y, x)
results = model.fit()
print('Beta: '+str(round(results.params[0],2)))

# How does this value compare to other sources such as Yahoo Finance (who uses a 5 year monthly model compared to 
# our 1 year daily)? Should this cause some skepticism when looking at results from models you didn't build?

## -------------PRACTICE-------------

1. Pull the 1 year price history using the scrub_price_hist function of any stock you like.

2. Clean up the output so it mirrors our output for Apple.

3.  Think of another market proxy besides the S&P 500, pull it's data, and clean it up.

4. Merge the two data sets, so that it mirrors the 'consol_data' variable.  Then merge in the new proxy data and SPY data.

5. Calculate the beta of your new stock relative to both SPY and your new market proxy.  Are they different significantly?
   Consider why they may be different.

## Another common measure of risk: Volatility
Much like beta, annual standard deviation of returns (more commonly known as volatility), can be a helpful tool to guage the relative risk a particular asset poses to a portfolio.  Let's see how we might go about calculating that for Apple.

In [ ]:
#Start with our Aaple data
aapl_data.head()

In [ ]:
# Remember that the covariance of a random variable with itself is the same as its variance.
aapl_vol = math.sqrt(pd.DataFrame([aapl_data['AAPL_Return_%'],aapl_data['AAPL_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('Apple Volatility: '+str(round(aapl_vol*100,2))+'%')

In [ ]:
# How does this compare to SPY?
spy_vol = math.sqrt(pd.DataFrame([spy_data['SPY_Return_%'],spy_data['SPY_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('SPY Volatility: '+str(round(spy_vol*100,2))+'%')

Are these results surprising?  Think about the implications of an annual volatility of over 20%, and how such a security would
behave if it fell out of the market's favor.

## Fundamental Data
Many critics argue any form of technical analysis is a waste of time and should be avoided.  Instead, they will argue for
fundament analysis, or effectively looking at various accounting measures to understand the health of a firm, and judge it's 
performance relative to its peers.  

In [ ]:
# This cell pulls the fundamental financial data from TD Ameritrade, cleans it up into a useable format and spits it out 
# in a nice table format using the Pandas package.  Note that book value per share is not
# entirely accurate, all other metrics have been vetted. Let's look at the whole S&P 500.
tickers =['AAPL','MSFT','AMZN','FB','GOOGL','GOOG','BRK.B','JNJ','V','PG','JPM','NVDA','UNH','MA','HD','VZ','DIS','NFLX',
          'ADBE','PYPL','CRM','MRK','T','INTC','PFE','CMCSA','BAC','WMT','KO','PEP','ABT','CSCO','XOM','TMO','ABBV','MCD',
          'COST','ACN','CVX','AMGN','MDT','AVGO','NKE','NEE','BMY','UNP','LIN','LLY','TXN','QCOM','DHR','PM','LOW','HON',
          'ORCL','UPS','AMT','IBM','C','SBUX','LMT','MMM','AMD','FIS','CHTR','WFC','RTX','BA','BLK','NOW','INTU','SPGI',
          'ISRG','MDLZ','GILD','CAT','MO','CVS','BKNG','ZTS','PLD','TGT','BDX','AXP','ANTM','VRTX','TJX','CCI','DE','D',
          'EQIX','TMUS','APD','CL','CI','SYK','GS','MS','CME','DUK','ATVI','ADP','BSX','MMC','CSX','REGN','SO','ITW','CB',
          'ICE','SHW','PGR','FISV','HUM','GE','NSC','NEM','FDX','NOC','AMAT','TFC','EW','KMB','USB','MU','GPN','ILMN',
          'ADSK','ECL','DG','EL','PNC','AON','MCO','BIIB','LRCX','WM','DD','ADI','BAX','ROP','GM','EMR','ETN','SCHW','AEP',
          'LHX','DLR','COP','EA','XEL','DXCM','GIS','CTSH','DOW','EXC','ORLY','HCA','SRE','GD','SBAC','EBAY','ROST','CNC',
          'CMG','COF','PSA','TEL','STZ','IDXX','APH','INFO','JCI','SYY','CMI','SNPS','TWTR','MNST','WEC','MET','A','TROW',
          'BK','VRSK','TRV','ES','MSCI','ALL','PCAR','PPG','IQV','ZBH','AZO','YUM','MAR','CDNS','HPQ','TT','KR','KLAC','F',
          'PRU','WBA','CLX','AFL','CTAS','BLL','ANSS','PEG','MSI','WLTW','PH','HLT','KMI','ROK','SLB','PSX','MCHP','AWK',
          'WELL','ADM','AIG','TDG','FAST','WMB','OTIS','GLW','RMD','MCK','ED','SWK','MKC','CARR','BBY','ALXN','XLNX','EOG',
          'PAYX','STT','MTD','DHI','FCX','DTE','CHD','ALGN','AVB','O','AME','SWKS','VFC','FTV','PPL','APTV','HSY','SPG',
          'CERN','LUV','DLTR','CTVA','MPC','CPRT','WY','LEN','ARE','VRSN','EQR','VLO','EFX','KHC','RSG','FLT','EIX','ETR',
          'WST','AJG','AEE','DAL','ODFL','FRC','AMP','TSN','LYB','KSU','MXIM','TFX','LVS','TTWO','KEYS','NTRS','DFS','KMX',
          'CMS','AMCR','CAG','MKTX','LH','AKAM','K','VTR','TSCO','VMC','CDW','CTXS','CBRE','INCY','VAR','FE','DOV','PXD',
          'COO','FTNT','VIAC','GWW','BR','PEAK','FITB','HOLX','MAS','EXPD','IP','XYL','NDAQ','DPZ','ESS','CAH','BF.B',
          'GRMN','ABC','DGX','HIG','GPC','EXR','HRL','DRE','NUE','SYF','EXPE','FMC','NVR','MTB','STE','ZBRA','ULTA','PAYC',
          'QRVO','IEX','MAA','SIVB','TIF','HAL','TYL','IFF','CHRW','SJM','MLM','WAT','LNT','CE','URI','BXP','PKI','KEY',
          'WAB','NLOK','HPE','RCL','JKHY','ABMD','CINF','EVRG','IR','J','HES','ETFC','OMC','DRI','CFG','JBHT','LDOS','TDY',
          'ATO','AES','MGM','IT','FBHS','ANET','RF','PHM','PFG','OKE','WDC','EMN','CNP','STX','WHR','AAP','NTAP','BIO',
          'CTL','UDR','HBAN','CBOE','HAS','ALB','LKQ','OXY','PKG','FOXA','XRAY','AVY','CXO','WU','UAL','HSIC','ALLE','BKR',
          'RJF','WRB','LW','TXT','UHS','CCL','L','IRM','BWA','RE','DISH','HST','PNW','GL','WRK','LYV','NI','SNA','NRG',
          'MYL','WYNN','CPB','JNPR','DVA','ROL','PNR','COG','PWR','FFIV','AIZ','LNC','TAP','CF','REG','AAL','IPG','HWM',
          'PRGO','AOS','LB','DISCK','NWL','MOS','RHI','SEE','BEN','HII','CMA','IPGP','VNO','HBI','FRT','MHK','LEG','ZION',
          'NWSA','AIV','NLSN','FANG','ALK','DXC','KIM','PVH','APA','NCLH','PBCT','FLIR','NBL','FOX','NOV','TPR','IVZ',
          'UNM','SLG','RL','DVN','GPS','FLS','MRO','KSS','DISCA','XRX','HFC','FTI','HRB','UAA','UA','NWS','COTY']
fundamental_data=scrub_fundamental_data(tickers,key)
fundamental_data.T

Think back to other accounting and finance courses, and assess whether you think the operating metrics of these companies seem to portray a healthy company, or not, and if they seem overvalued based on PE or PS measures.  Remember, there really is no right or wrong answer here.  

## Get Annual Return Data
The below cell pulls the annual return data from TD Ameritrade, cleans it up into a useable format and spits it out in a nice table format using the Pandas package.

In [ ]:
annual_returns_df=get_annual_returns(tickers,key)
annual_returns_df

## Fundamental Data Frame Info
The next few code blocks give us information on how our data is structered, the columns it contains, the data types of the columns (string, integer, float, etc.), summary statistics, null values, etc.

In [ ]:
for col in fundamental_data.columns:
    try:
        fundamental_data[col]=fundamental_data[col].astype('float')
    except:
        pass

#Data columns
fundamental_data.columns

In [ ]:
#General data statistics
fundamental_data.describe()

In [ ]:
# Data Frame information (null, data type etc)
fundamental_data.info()

In [ ]:
# Uncorrelated features are generally more powerful predictors
# This chart will let us see wich fundamental factors may be most meaningful in predicting returns
colormap = plt.cm.viridis
plt.figure(figsize=(40, 40))
plt.title('Pearson Correlation of Features', y=1.05, size=14)
sns.set(font_scale=1.4)
sns.heatmap(fundamental_data.corr().round(2)\
            ,linewidths=0.1,vmax=1.0, square=True, cmap=colormap, \
            linecolor='white', annot=True);

## -------------PRACTICE-------------

1. Pick a stock we haven't looked at and grab its fundamental data.

2. Calculate the annual volatility of TQQQ.

3. Find the beta to SPY of AMD as well as its volatility and make a conclusion about its risk profile.

4. Get the fundamental data of AMC.  

## Linear Regression
Linear regression models are a great way to test if ratios are even worth looking at.  Lets check based on the stocks we picked above

In [ ]:
x = fundamental_data[['epsTTM', 'grossMarginMRQ', 'grossMarginTTM','interestCoverage',
                      'netProfitMarginMRQ', 'netProfitMarginTTM','operatingMarginMRQ', 'operatingMarginTTM','peRatio', 'pegRatio', 'prRatio', 'quickRatio', 'returnOnAssets',
                      'returnOnEquity', 'returnOnInvestment']] # Independent variable
y = fundamental_data["Annual_Return_Percent"] # Response / Target Variable

print(x.shape, y.shape)
x=sm.add_constant(x)
result=sm.OLS(y,x).fit()
print(result.summary())

## Regression Results
The above regression results show us that by looking at a handful of common fundamental metrics, we are able to explain only 7% of the movement in the stocks analyzed (per R-squared).  Additionally, it is usefuly to look at the Prob (F-statistic) and Prob(Jarque-Bera) as these two statistical tests are crucial gauges of model efficacy.  Specifically, if both values are under .05, we can reasonable assume the variables used in the model are meaningful indicators. 

While these results suggest maybe fundamental analysis isn't the best predictor, it may also be a signal that something unrelated to the companies themselves is driving performance.  

## Remark: Volatility doesn't live in a vacuum
In our previous exploration of volatility, we only examined the volatility of individual assets.  While this is certainly useful information, understanding how assets behave together can be even more insightful.  After all, who only invests in a single asset?

Unlike using a weighted average to quickly find a portfolios return, we have to use the square root of the following formula to gauge a portfolio's volatility.  This is incredibly important, as often times, a portfolio can be made up of extremely volatile assets, but, because of their relationships, the net volatility is relatively low.

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/c91273fd3499f6172ed9baf853d3d3ae8d02c62d">


In [ ]:
#Let's use the formula above to find the volatility of a portfolio of half AMD and half AMC

amd_weight = .5
amc_weight = .5
tickers = ['AMD','AMC']

###Grab Price History
price_hist = scrub_price_hist(tickers,key)
price_hist = price_hist[['close','ticker','datetime']]


###Calculate Portfolio Volatility
count = 0
for ticker in tickers:
    if count == 0:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = temp
        count = 1
    else:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = pd.merge(master,temp,how = 'left',on = ['datetime'])


consol_returns = pd.DataFrame([master[tickers[0] + '_Return_%']*amd_weight,master[tickers[1] + '_Return_%']*amc_weight]).T
port_volatility = math.sqrt(consol_returns.cov().to_numpy().sum())*math.sqrt(252)

###Calculate Individual Security Volatilities
sec_vols = []        
for ticker in tickers:
    temp = [ticker,(math.sqrt(252)*math.sqrt(statistics.variance(master[ticker + '_Return_%'])))]
    sec_vols.append(temp)
    
    
for i in range (0,len(tickers)):
    print(tickers[i] + ' Volatility: ',round(sec_vols[i][1]*100,2),'%')
print('Portfolio Volatility: ',round(port_volatility*100,2),'%')

Notice how the volatility of the portfolio is significantly less than had you only held AMC.  Conversely,  consider the implications of the volatility of these holdings. 

## -------------PRACTICE-------------

1. Find the annualized volatility of an equally weighted portfolio of AMGN and PG.  Assume 252 trading days in a year and use one year of historicals.

2. Find the annualized volatility of AMGN and PG respecively with the same assumptions as question 1.

3. Find the volatility of XRX using data from June of 2021 only.  Compare it to its volatility in September of 2021.

## TD API: A Risk Management Study
Understanding the risk of a portfolio of stocks is often times just as crucial, if not more so, than understanding its growth opportunities.  In this exercise we will examine a portfolio that has equal weights in APPL and TSLA, and attempt to gauge how much capital we are putting at risk by holding this portfolio.  To keep math simple, we will assume that the portfolio is valued at $50,000.  

## Value at Risk
Value at Risk, or simply VaR, has been around for decades, and is considered a gold standard for the benchmark of porfolio risk.  Barring any significant shifts in company health or direction, VaR can be thought of as largest paper loss an investor will see over a predefined time horizon by holding a security(s).  We will look at one month VaR, and choose a confidence level of 95% in order to have a high degree of confidence that we are in fact looking at a reasonable worst case.  While it can be beneficial to look at the 99% case, 95% will be a good start.  With our parameters defined, VaR can be calculated as follows:

VaR = Portfolio Volatility * Portfolio Value * 1.65

note that the 1.65 translates to a 95% confidence interval's z-score

In [ ]:
###Define Portfolio Inputs
aapl_weight = .5
tsla_weight = .5
portfolio_value = 50000
days_into_future = 21  ##remember, a typical month only has 21 trading days
tickers = ['TSLA','AAPL']

In [ ]:
#Let's calculate our portfolio volatility
###Grab Price History
price_hist = scrub_price_hist(tickers,key)
price_hist = price_hist[['close','ticker','datetime']]


###Calculate Portfolio Volatility
count = 0
for ticker in tickers:
    if count == 0:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = temp
        count = 1
    else:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = pd.merge(master,temp,how = 'left',on = ['datetime'])


consol_returns = pd.DataFrame([master[tickers[0] + '_Return_%']*tsla_weight,master[tickers[1] + '_Return_%']*aapl_weight]).T
port_volatility = math.sqrt(consol_returns.cov().to_numpy().sum())*math.sqrt(days_into_future)

###Calculate Individual Security Volatilities
sec_vols = []        
for ticker in tickers:
    temp = [ticker,(math.sqrt(days_into_future)*math.sqrt(statistics.variance(master[ticker + '_Return_%'])))]
    sec_vols.append(temp)



###Compare individual security volatility to Portfolio Volatility
for i in range (0,len(tickers)):
    print(tickers[i] + ' Volatility: ',round(sec_vols[i][1]*100,2),'%')
print('Portfolio Volatility: ',round(port_volatility*100,2),'%')


In [ ]:
###Calculate VaR
VaR = portfolio_value * 1.65 * port_volatility
print('$',round(VaR,2))


## Conclusions

By holding a \\$50,000 portfolio that is equally weighted in AAPL and TSLA stock, an investor should be comfortable with a significant likelihood of losing a significant amount of capital over the course of one month.  

## -------------PRACTICE-------------

1.  Find the 1-year value at risk for a portfolio of 25% FB and 75% MO.  Assume 252 trading days in a year, and use the most recent 1 year pricing data.

## -------------EXTEND YOUR UNDERSTANDING-------------

Assume your broker allows you to invest on margin with no cost of borrowring up to 30% (i.e. for each \\$1 you have, you can invest \\$1.30).  Assume you have $10,000, want to use all the margin, and are feeling very good about an equally weighted portfolio of AIV and AIRC.  What is your one-month VaR?  Assume 21 trading days in a month, and use the most recent year's worth of data to estimate volatility.  

## Further Readings

https://www.investopedia.com/financial-edge/0113/has-high-frequency-trading-ruined-the-stock-market-for-the-rest-of-us.aspx

https://finmath.uchicago.edu/careers/career-paths-in-quantitative-finance/